import the necessary libraries

In [1]:

import numpy as np
import math
import os
import csv
import pandas as pd
import qalsadi.lemmatizer
import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras import layers
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D, Conv1D,MaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


Read the train data

In [2]:
headers = []
body = []
with open('processed_train.csv', 'r',encoding='utf-8') as reader:
    lines = reader.readlines()
    count = 0
    for line in lines:
        
        if(count%10000 ==0):
            print("Finished "+str(count)+ "lines")
        if(count == 0):
            headers.append(line.split(','))
        else:
            if line!='"\n':
                body.append(line.split(','))

        count += 1

df = pd.DataFrame(data = body,columns = headers[0])

Finished 0lines
Finished 10000lines
Finished 20000lines
Finished 30000lines
Finished 40000lines
Finished 50000lines
Finished 60000lines
Finished 70000lines
Finished 80000lines


In [3]:
df

,,,Context,Response,Text,"""ID\n"
0,0,0,عاطفي,هل كان هذا الصديق الذي كنت تحبه أم مجرد أفضل ص...,أتذكر ذهابي لمشاهدة الألعاب النارية مع أعز أصد...,"""0\n"
1,1,1,عاطفي,اين ذهبت؟,كان هذا أفضل صديق. اشتقت لها.,"""1\n"
2,2,2,عاطفي,هل كان هذا شيئًا حدث بسبب جدال؟,لم نعد نتحدث.,"""2\n"
3,3,3,خائف,أجل؟ أنا حقا لا أرى كيف,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,"""3\n"
4,4,4,خائف,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...,لا تشعر بذلك .. إنه لأمر عجيب,"""4\n"
...,...,...,...,...,...,...
40249,40249,40249,خائف,يا إلهي ، هذا مخيف جدًا ، أنا آسف جدًا لسماع ذلك.,اضطر منزلي الذي احترق في الليلة السابقة لإخراج...,"""40249\n"
40250,40250,40250,عاطفي,هل وجدت أي شيء عظيم؟,كنت أتصفح الأشياء في العلية الليلة الماضية,"""40250\n"
40251,40251,40251,عاطفي,يا لها من ذكرى رائعة.,نعم ، لقد وجدت بعض الصور القديمة عندما اعتدنا ...,"""40251\n"
40252,40252,40252,مندهش,يا هذا رائع! هذا رائع أليس كذلك؟,استيقظت هذا الصباح على زوجتي تخبرني أنها حامل!,"""40252\n"


Initialize the necessary variables for the lemmetization and check whether the lemmetization already exists or not.

In [4]:
data = df.values.tolist()
data_utterance_lemmetization = []
dict_word_counts = {}
count = 0
regex = re.compile(r'[\',a-z,A-Z,0-9]+')

isFile = os.path.isfile('Document_Frequencies.csv') 
print(isFile)

True


Create the lemmetization and document count files else read the files output is the variables are filled with the correct data.

In [5]:
if not isFile:
    lemmer = qalsadi.lemmatizer.Lemmatizer()
    for utterance in data:

        if(count%100 ==0):
            print('finished lemmetizing '+str(count)+' sentences')
        count+=1

        if utterance[4] != None or utterance[4] != 'Error':
            text = regex.sub('', utterance[4])
        else:
            text = []
    #     print(text)
        lemmas = lemmer.lemmatize_text(text, return_pos=True)

        lemmas = [a[0] for a in lemmas if len(a) == 2 and a[1]!="stopword" and a[1]!="all" and a[0] !='.']

        data_utterance_lemmetization.append(lemmas)
        for word in lemmas:
            if word in dict_word_counts:
                dict_word_counts[word] +=1
            else:
                dict_word_counts[word] = 1
    i = 0
    with open("Document_Frequencies.csv", 'w',newline='') as csvfile: 
        header = ['Terms','Document_Count']
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(header)
        # writing the fields 
        for key in dict_word_counts:
            if i%100 == 0:
                print('finished writing '+str(i)+' setences on the file')
            i+=1
            row = []
            row.append(key)
            row.append(dict_word_counts[key])
            csvwriter.writerow(row)
    print("done")
    i = 0
    with open("Document_Sentences.txt", 'w',newline='') as file: 
        header = ['index','tokenization']
        # creating a csv writer object 
        file.write('Index;Tokenization\n')
        # writing the fields 
        for sentence in data_utterance_lemmetization:
            if i%100 == 0:
                print('finished writing '+str(i)+' setences on the file')
            row = str(i)+';'
            for word in sentence:
                row+=word+','

            file.write(row+'\n')  
            i+=1
    print("done")
else:
    print("I am here")
    with open('Document_Frequencies.csv', 'r',encoding="utf8") as reader:
        lines = reader.readlines()
        count = 0
        for line in lines:
            if(count%100 ==0):
                print("Finished "+str(count)+ "lines")
            if(count != 0):
                count_words = int(line.split(',')[1])       
                dict_word_counts[line.split(',')[0]] = count_words
            count += 1
            
    with open('Document_Sentences.txt', 'r',encoding="utf8") as reader:
        lines = reader.readlines()
        count = 0
        for line in lines:
            if(count%100 ==0):
                print("Finished "+str(count)+ "lines")
            if(count != 0):
                without_line_breaks = line.replace("\n", "")
                if len(line.split(';')[1])>1:
                    without_line_breaks = without_line_breaks[:-1]
                    data_utterance_lemmetization.append(without_line_breaks.split(';')[1].split(','))
                else:
                    data_utterance_lemmetization.append([])
            count += 1
#     LOAD FILES AND DEAL WITH THE ENCODINGS
    
    

I am here
Finished 0lines
Finished 100lines
Finished 200lines
Finished 300lines
Finished 400lines
Finished 500lines
Finished 600lines
Finished 700lines
Finished 800lines
Finished 900lines
Finished 1000lines
Finished 1100lines
Finished 1200lines
Finished 1300lines
Finished 1400lines
Finished 1500lines
Finished 1600lines
Finished 1700lines
Finished 1800lines
Finished 1900lines
Finished 2000lines
Finished 2100lines
Finished 2200lines
Finished 2300lines
Finished 2400lines
Finished 2500lines
Finished 2600lines
Finished 2700lines
Finished 2800lines
Finished 2900lines
Finished 3000lines
Finished 3100lines
Finished 3200lines
Finished 3300lines
Finished 3400lines
Finished 3500lines
Finished 3600lines
Finished 3700lines
Finished 3800lines
Finished 3900lines
Finished 4000lines
Finished 4100lines
Finished 4200lines
Finished 4300lines
Finished 4400lines
Finished 4500lines
Finished 4600lines
Finished 4700lines
Finished 4800lines
Finished 4900lines
Finished 5000lines
Finished 5100lines
Finished 5200l

In [6]:
data_utterance_lemmetization[0]

['تذكر',
 'ذهاب',
 'مشاهد',
 'ألعاب',
 'نار',
 'عز',
 'أصدقاء',
 'مر',
 'أولى',
 'قضة',
 'قت',
 'مفرد',
 'مع',
 'رغم',
 'جود',
 'كثير',
 'أشخاص',
 'شعر',
 'أشخاص',
 'وحيد',
 'عالم']

In [7]:
print(len(dict_word_counts),len(data_utterance_lemmetization),len(body)) 

7558 40254 40254


Get the total number of documents and remove the empty cells from the count

In [8]:
counter = 0
for x in data_utterance_lemmetization:
    if x ==[]:
        counter+=1
print(counter)
total_number_documents = len(data_utterance_lemmetization)-counter
print(total_number_documents)

525
39729


make an array of normalized tf_idf for each document

In [9]:
tf_idf = []
for sentence in data_utterance_lemmetization:
    term_frequency = {}
    tf_idf_words ={}
    for word in sentence:
        if word in term_frequency:
            term_frequency[word] +=1
        else:
            term_frequency[word] = 1
    for word in term_frequency:
        tf_idf_words[word] = (1+math.log(term_frequency[word]))* (math.log(total_number_documents/dict_word_counts[word]))
    normalization_value = 0
    for word in tf_idf_words:
        normalization_value += tf_idf_words[word]**2
    normalization_value = math.sqrt(normalization_value)
    
    for word in tf_idf_words:
        tf_idf_words[word] = tf_idf_words[word] / normalization_value
        
    tf_idf.append(tf_idf_words)

In [10]:
tf_idf[0]

{'تذكر': 0.19697797096916786,
 'ذهاب': 0.17288693587530346,
 'مشاهد': 0.21570034009722022,
 'ألعاب': 0.23922837634965943,
 'نار': 0.26346305705048984,
 'عز': 0.2523956505305077,
 'أصدقاء': 0.15869903565113733,
 'مر': 0.1728274153034388,
 'أولى': 0.21084671255004925,
 'قضة': 0.26915997438471645,
 'قت': 0.23451291417297054,
 'مفرد': 0.23209987482781091,
 'مع': 0.23140149570200128,
 'رغم': 0.1820047139888703,
 'جود': 0.2328097346713066,
 'كثير': 0.12588548458957402,
 'أشخاص': 0.3705680398067583,
 'شعر': 0.09985551112189194,
 'وحيد': 0.24268479400367984,
 'عالم': 0.22517593156651244}

functions to process the query to the desired structure

In [11]:
def process_query(text):
    lemmer = qalsadi.lemmatizer.Lemmatizer()
    regex = re.compile(r'[\',a-z,A-Z,0-9]+')
    lemmas = lemmer.lemmatize_text(text, return_pos=True)
    lemmas = [a[0] for a in lemmas if len(a) == 2 and a[1]!="stopword" and a[1]!="all" and a[0] !='.']
    term_frequency = {}
    tf_idf_words ={}
    for word in lemmas:
        if word in term_frequency:
            term_frequency[word] +=1
        else:
            term_frequency[word] = 1
            
    for word in term_frequency:
        tf_idf_words[word] = (1+math.log(term_frequency[word]))* (math.log(total_number_documents/dict_word_counts[word]))
    normalization_value = 0
    for word in tf_idf_words:
        normalization_value += tf_idf_words[word]**2
    normalization_value = math.sqrt(normalization_value)
    
    for word in tf_idf_words:
        tf_idf_words[word] = tf_idf_words[word] / normalization_value
    return tf_idf_words

In [12]:
def cosine_similarity(query):
    text = process_query(query)
    max_index = -1
    max_similarity = 0
    index = 0
    for dictonary in tf_idf:
        similarity = 0
        for key in text:
            if key in dictonary:
                similarity+= text[key] *dictonary[key]
        if similarity >max_similarity:
            max_index = index
            max_similarity = similarity
        index+=1
    if max_index!= -1:
        return max_similarity,body[max_index][3],body[max_index][2]
    return 0,"I couldn't hear you can you please say it again differently"    

test string you can copy paste the string to try the chatbot... make sure to remove the string quations ('')

run this cell for the chatbot and when you want to exit the chatbot write exit

In [146]:
while(True):
    query = input()
    if query != "exit":
        similarity,answer,context = cosine_similarity(query)
        print(answer,' the context is ',context)
    else:
        print("pleased to talk to you, goodbye")
        break


أتذكر ذهابي لمشاهدة الألعاب النارية مع أعز أصدقائي. كانت هذه هي المرة الأولى التي قضينا فيها وقتًا بمفردنا معًا. على الرغم من وجود الكثير من الأشخاص ، إلا أننا شعرنا بأننا الأشخاص الوحيدون في العالم.
هل كان هذا الصديق الذي كنت تحبه أم مجرد أفضل صديق؟  the context is  عاطفي
exit
pleased to talk to you, goodbye


In [13]:
y = df['Context']

In [14]:
y = np.array(list(map(lambda x: 1 if x == "معد"or x == "فخور" or 
                      x == "مخلص"or x == "عاطفي" or x == "ممتن" or 
                      x == "فرح" or x == "حنين" or x == "موثوق" or 
                      x == "الثقة" or x == "رعاية" or x == "متفائل" or 
                      x == "المحتوى" or x == "تأثرت" or x == "بهيجة" or 
                      x == "مندهش" or x == "توقع" else 0, y)))

In [15]:
text = []
for i in range(len(data_utterance_lemmetization)):
    text.append(' '.join(data_utterance_lemmetization[i]))
    
from keras.utils import to_categorical
y = to_categorical(y)

In [16]:
print(len(y),len(text))


40254 40254


In [17]:

max_words = 7558
vocab_size = 7558
max_len = 25 


tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text)
text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, padding='post', maxlen=max_len)



X_train, X_test, Y_train, Y_test = train_test_split(text, y, test_size =.2, random_state=100)

In [34]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint


model1 = Sequential()
model1.add(layers.Embedding(max_words, 25)) #The embedding layer
model1.add(layers.LSTM(15,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(2,activation='sigmoid'))


model1.summary()

# model0 = Sequential()
# model0.add(layers.Embedding(max_words, 15))
# model0.add(layers.SimpleRNN(15,return_sequences=True))
# model0.add(layers.SimpleRNN(15))
# model0.add(layers.Dense(2,activation='softmax'))

# model0.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 25)          188950    
_________________________________________________________________
lstm_2 (LSTM)                (None, 15)                2460      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 32        
Total params: 191,442
Trainable params: 191,442
Non-trainable params: 0
_________________________________________________________________


In [35]:
model1.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)

In [36]:
history = model1.fit(X_train, Y_train, epochs=10,validation_data=(X_test, Y_test),callbacks=[checkpoint1])

Train on 32203 samples, validate on 8051 samples
Epoch 1/10
32203/32203 [==============================] - 14s 434us/step - loss: 0.5880 - accuracy: 0.6801 - val_loss: 0.5416 - val_accuracy: 0.7312

Epoch 00001: val_accuracy improved from -inf to 0.73115, saving model to best_model1.hdf5
Epoch 2/10
32203/32203 [==============================] - 14s 441us/step - loss: 0.5140 - accuracy: 0.7470 - val_loss: 0.5283 - val_accuracy: 0.7318

Epoch 00002: val_accuracy improved from 0.73115 to 0.73177, saving model to best_model1.hdf5
Epoch 3/10
32203/32203 [==============================] - 12s 370us/step - loss: 0.4978 - accuracy: 0.7626 - val_loss: 0.5160 - val_accuracy: 0.7503

Epoch 00003: val_accuracy improved from 0.73177 to 0.75028, saving model to best_model1.hdf5
Epoch 4/10
32203/32203 [==============================] - 12s 382us/step - loss: 0.4901 - accuracy: 0.7647 - val_loss: 0.5136 - val_accuracy: 0.7506

Epoch 00004: val_accuracy improved from 0.75028 to 0.75059, saving model to

In [37]:
loss, acc = model1.evaluate(X_test, Y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 75.400573


In [56]:
def process_context_query(text):
    lemmer = qalsadi.lemmatizer.Lemmatizer()
    regex = re.compile(r'[\',a-z,A-Z,0-9]+')
    lemmas = lemmer.lemmatize_text(text, return_pos=True)
    lemmas = [a[0] for a in lemmas if len(a) == 2 and a[1]!="stopword" and a[1]!="all" and a[0] !='.']
    sentence = ' '.join(lemmas)
    sentence = tokenizer.texts_to_sequences([sentence])
    sentence = pad_sequences(sentence, padding='post', maxlen=max_len)
    return sentence

query = process_context_query('لقد فقدت وظيفتي العام الماضي وغضبت حقًا.')  
y_pred = model1.predict(query)
query

array([[  1, 211,  59,  27,  22, 522,   6,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [57]:
np.argy_pred

array([[0.94546413, 0.05570153]], dtype=float32)

In [58]:
y[14]

array([1., 0.], dtype=float32)

In [55]:
body[14]

['14',
 '14',
 'غاضب',
 'أنا آسف لسماع هذا. هل حدث ذلك فجأة؟',
 'لقد فقدت وظيفتي العام الماضي وغضبت حقًا.',
 '"14\n']